In [1]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
df_airports = pd.read_csv('airports.csv')

In [3]:
df_airlines = pd.read_csv('airlines.csv')

In [32]:
df_hdata = pd.read_csv('historic_data.csv', low_memory=False)

In [34]:
df_hdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458628 entries, 0 to 2458627
Data columns (total 25 columns):
 #   Column               Dtype  
---  ------               -----  
 0   SCHEDULED_DEPARTURE  object 
 1   DEPARTURE_TIME       object 
 2   SCHEDULED_ARRIVAL    object 
 3   ARRIVAL_TIME         object 
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   DEPARTURE_DELAY      float64
 10  TAXI_OUT             float64
 11  WHEELS_OFF           float64
 12  SCHEDULED_TIME       float64
 13  ELAPSED_TIME         float64
 14  AIR_TIME             float64
 15  DISTANCE             int64  
 16  WHEELS_ON            float64
 17  TAXI_IN              float64
 18  CANCELLED            int64  
 19  CANCELLATION_REASON  object 
 20  AIR_SYSTEM_DELAY     float64
 21  SECURITY_DELAY       float64
 22  AIRLINE_DELAY        float64
 23  LATE_AIRCRAFT_DELAY  float64
 24

In [5]:
df_validate = pd.read_csv("future_data.csv", low_memory=False)

In [31]:
df_hdata['SCHEDULED_DEPARTURE'] = pd.to_timedelta(df_hdata['SCHEDULED_DEPARTURE'], 
                            
                            errors='coerce')
df_hdata['DEPARTURE_TIME'] = pd.to_timedelta(df_hdata['DEPARTURE_TIME'], 
                            errors='coerce')
df_hdata['SCHEDULED_ARRIVAL'] = pd.to_timedelta(df_hdata['SCHEDULED_ARRIVAL'], 
                            errors='coerce')
df_hdata['ARRIVAL_TIME'] = pd.to_timedelta(df_hdata['ARRIVAL_TIME'], 
                            errors='coerce')
df_hdata['SCHEDULED_TIME'] = pd.to_timedelta(df_hdata['SCHEDULED_TIME'], 
                            errors='coerce')

TypeError: dtype datetime64[ns] cannot be converted to timedelta64[ns]

In [7]:
import matplotlib.pyplot as plt

In [8]:
df_hdata[df_hdata.CANCELLED ==1]

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_ARRIVAL,ARRIVAL_TIME,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,...,DISTANCE,WHEELS_ON,TAXI_IN,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
2,2019-03-01 00:01:00,NaT,0 days 01:53:00,NaT,EV,4682,N15910,ORD,BHM,NaN,...,584,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
24,2019-03-01 00:07:00,0 days,0 days 02:35:00,NaT,OO,6315,N745SK,DEN,ICT,-7.0,...,420,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
29,2019-03-01 00:08:00,NaT,0 days 02:39:00,NaT,OO,6378,N957SW,DEN,LBB,NaN,...,456,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
37,2019-03-01 00:11:00,NaT,0 days 02:59:00,NaT,UA,1881,NaN,ORD,IAD,NaN,...,588,NaN,NaN,1,A,NaN,NaN,NaN,NaN,NaN
38,2019-03-01 00:11:00,NaT,0 days 03:38:00,NaT,UA,1489,NaN,DEN,MEM,NaN,...,872,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458496,2019-07-31 22:59:00,NaT,0 days 00:25:00,NaT,B6,2002,N565JB,JFK,BUF,NaN,...,301,NaN,NaN,1,C,NaN,NaN,NaN,NaN,NaN
2458519,2019-07-31 23:06:00,NaT,0 days 07:40:00,NaT,B6,670,N612JB,SJC,JFK,NaN,...,2569,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
2458520,2019-07-31 23:09:00,NaT,0 days 05:00:00,NaT,B6,994,N595JB,DEN,BOS,NaN,...,1754,NaN,NaN,1,C,NaN,NaN,NaN,NaN,NaN
2458558,2019-07-31 23:29:00,NaT,0 days 02:16:00,NaT,B6,551,N589JB,TPA,SJU,NaN,...,1237,NaN,NaN,1,A,NaN,NaN,NaN,NaN,NaN


In [45]:
df_hdata[df_hdata['CANCELLED'] ==1].groupby('AIRLINE').size().nlargest(5)

AIRLINE
WN    6508
EV    5811
MQ    4686
OO    4623
AA    4043
dtype: int64

In [51]:
delay = df_hdata[df_hdata['ARRIVAL_TIME'].notnull()]

In [52]:
delay[delay['ARRIVAL_TIME'] > delay['SCHEDULED_TIME']].groupby('AIRLINE').size().nlargest(5)

AIRLINE
WN    529941
DL    375048
AA    314913
OO    242845
EV    227583
dtype: int64

In [29]:
plt.bar(df_hdata['AIRLINE'], count(df_hdata['CANCELLED']))

NameError: name 'count' is not defined

In [10]:
#dropping the rows with cancelled flights
df_hdata=df_hdata[df_hdata.CANCELLED ==0]

In [11]:
df_hdata=df_hdata.drop(columns=['DEPARTURE_TIME', 'DEPARTURE_DELAY', 'CANCELLED', 'CANCELLATION_REASON', 'TAXI_OUT', 'WHEELS_OFF', 'ELAPSED_TIME', 'AIR_TIME','WHEELS_ON', 'TAXI_IN', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'])

In [12]:
df_hdata=df_hdata.dropna(subset=['ARRIVAL_TIME'])

In [13]:
df_hdata.isna().sum()

SCHEDULED_DEPARTURE    0
SCHEDULED_ARRIVAL      0
ARRIVAL_TIME           0
AIRLINE                0
FLIGHT_NUMBER          0
TAIL_NUMBER            0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_TIME         0
DISTANCE               0
dtype: int64

In [25]:
type(df_hdata['ARRIVAL_TIME'])

pandas.core.series.Series

In [ ]:
import datetime as dt
data_df['Date'] = pd.to_datetime(data_df['Date'])
data_df['Date']=data_df['Date'].map(dt.datetime.toordinal)

In [22]:
df_hdata['ARRIVAL_TIME'] = pd.to_datetime(df_hdata['ARRIVAL_TIME']) 
df_hdata['SCHEDULED_ARRIVAL']= pd.to_datetime(df_hdata['SCHEDULED_ARRIVAL'])

TypeError: dtype timedelta64[ns] cannot be converted to datetime64[ns]

In [19]:
df_hdata

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE
0,2019-03-01 00:01:00,0 days 01:37:00,0 days 01:20:00,UA,1238,N812UA,DEN,BIL,0 days 00:00:00.000000096,455
1,2019-03-01 00:01:00,0 days 01:33:00,0 days 01:52:00,EV,6134,N14171,ORD,BNA,0 days 00:00:00.000000092,409
3,2019-03-01 00:02:00,0 days 03:04:00,0 days 02:56:00,UA,1550,N822UA,ORD,BTV,0 days 00:00:00.000000122,763
4,2019-03-01 00:02:00,0 days 03:22:00,0 days 03:30:00,UA,1502,N33262,ORD,BOS,0 days 00:00:00.000000140,867
5,2019-03-01 00:02:00,0 days 01:54:00,0 days 02:01:00,UA,1456,N808UA,DEN,BZN,0 days 00:00:00.000000112,524
...,...,...,...,...,...,...,...,...,...,...
2458623,2019-07-31 23:59:00,0 days 05:39:00,0 days 05:22:00,AA,1496,N3MJAA,LAS,ORD,0 days 00:00:00.000000220,1514
2458624,2019-07-31 23:59:00,0 days 05:55:00,0 days 06:42:00,AA,1283,N3GEAA,SEA,ORD,0 days 00:00:00.000000236,1721
2458625,2019-07-31 23:59:00,0 days 06:00:00,0 days 07:03:00,AA,219,N3LKAA,LAX,ORD,0 days 00:00:00.000000241,1744
2458626,2019-07-31 23:59:00,0 days 03:47:00,0 days 04:59:00,B6,1503,N592JB,JFK,SJU,0 days 00:00:00.000000228,1598


In [14]:
Y = df_hdata['ARRIVAL_TIME']
X = df_hdata.loc[:, df_hdata.columns!='ARRIVAL_TIME']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [16]:
categorical_cols = ['AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']

In [29]:
type(y_train)

pandas.core.series.Series

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
model1 = LinearRegression().fit(X_train, y_train)

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [9]:

one_hot_columns = pd.get_dummies(X_train[categorical_cols])
one_hot_columns.head()

NameError: name 'X_train' is not defined

# Data Preprocessing with Pandas
Preprocessing is the process of doing a pre-analysis of data, in order to transform them into a standard and normalized format.
Preprocessing involves the following aspects:
- missing values
- data formatting
- data standardization
- data normalization
- data binning


#Missing Values

In [ ]:
X_train.isna().sum()

In [ ]:
df_hdata.shape

#Drop Missing Values

In [ ]:
drop_data = df_hdata.dropna(thresh=0.8*len(df_hdata), axis = 1)
#axis=0 drop row while axis=1 drop coulm 
#inplace = True to store changes in the original dataframe

In [ ]:
print('Dimensions of the dataset:',df_hdata.shape)
print('Dimensions after removing columns:',drop_data.shape)

#Replace missing values

In [ ]:
df_hdata.dtypes

#Numerical Columns

In [ ]:
import numpy as np
numeric = df_hdata.select_dtypes(include=np.number)
numeric_columns = numeric.columns
print(numeric_columns)

In [ ]:
df_rp= df_hdata
df_rp[numeric_columns]= df_rp[numeric_columns].fillna(df_rp[numeric_columns].mean())
#replace numeric data with its column mean

In [ ]:
df_rp.isna().sum()

#categorical columns

In [ ]:
boolean_columns = df_rp.select_dtypes(include=object).columns.tolist()

print(boolean_columns)
df_rp[boolean_columns] = df_rp[boolean_columns].astype('bool')

In [ ]:
df_rp[boolean_columns]= df_rp[boolean_columns].fillna(df_rp[boolean_columns].mode())

In [ ]:
df_rp.isna().sum()

In [ ]:
df_rp.head()

In [ ]:
df_rp[df_rp.duplicated(keep=False)]